In [ ]:
import json
from datetime import datetime

# 讀取 JSON 檔案
with open(r'C:\Users\student\Desktop\disney\history_test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 新增一個鍵值對，儲存星期幾
for record in data:
    datetime_str = record['datetime']
    dt = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M')
    record['weekday'] = dt.strftime('%A')

# 輸出結果
for record in data:
    print(f"日期: {record['datetime']}, 星期: {record['weekday']}")

# 如果需要將結果保存到新的 JSON 檔案中
with open(r'C:\Users\student\Desktop\disney\save_time.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [2]:
# 導入必要的庫
import pandas as pd
import json
from datetime import timedelta

# 讀取 JSON 檔案
file_path = '/Users/chianlee/Desktop/disney/data/testing_data_sea.json'
file_path_names = '/Users/chianlee/Desktop/disney/data/sea_namelist.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with open(file_path_names, 'r', encoding='utf-8') as file:
    names = json.load(file)

In [3]:
# 轉換為 DataFrame
df_data = pd.DataFrame(data)
df_names = pd.DataFrame(names)

# 刪除 'FacilityName' 欄位並移除 'StandbyTime' 為 null 的行
df_data = df_data.drop(columns=['FacilityName'])
df_data = df_data[df_data['StandbyTime'].notna()]
df_data = df_data[df_data['StandbyTime'] != False]

# 顯示處理後的5筆數據
df_data.head(5)

,StandbyTime,FacilityID,UpdateTime,datetime
0,5,234,9:00,2024-07-31 07:36
3,5,222,9:00,2024-07-31 07:36
4,5,236,9:00,2024-07-31 07:36
5,0,235,9:00,2024-07-31 07:36
6,5,220,9:00,2024-07-31 07:36


In [ ]:
# 解析日期時間欄位
df_data['UpdateTime'] = pd.to_datetime(df_data['UpdateTime'], format='%H:%M')
df_data['datetime'] = pd.to_datetime(df_data['datetime'])

# 將所有 datetime 欄位加上一小時
df_data['datetime'] = df_data['datetime'] + pd.Timedelta(hours=1)

# 提取日期並應用於 UpdateTime
df_data['UpdateTime'] = df_data.apply(
    lambda row: row['UpdateTime'].replace(year=row['datetime'].year, month=row['datetime'].month, day=row['datetime'].day),
    axis=1
)

# 顯示處理後的5筆數據
df_data.head(-5)

In [ ]:
# 計算 UpdateTime 和 datetime 之間的時間差（以小時為單位）
df_data['time_difference'] = (df_data['datetime'] - df_data['UpdateTime'])

# 刪除時間差超過2小時的資料
df_filtered = df_data[df_data['time_difference'] <= timedelta(hours=2)]

# 刪除多餘的時間差欄位
df_filtered = df_filtered.drop(columns=['time_difference'])

# 顯示處理後的5筆數據
df_filtered.head(-5)

In [ ]:
# 將 StandbyTime 轉換為數值型
df_filtered['StandbyTime'] = pd.to_numeric(df_filtered['StandbyTime'], errors='coerce')

# 提取設施名稱和小時
df_filtered['Hour'] = df_filtered['datetime'].dt.hour

# 篩選 8 點到 20 點之間的數據
df_filtered = df_filtered[(df_filtered['Hour'] >= 8) & (df_filtered['Hour'] <= 20)]

# 合併數據框以添加英文設施名稱
df_filtered = df_filtered.merge(df_names[['FacilityID', 'FacilityEnglish']], on='FacilityID', how='left')

# 確保每個設施在 8 點到 20 點都有數據
all_hours = pd.DataFrame({'Hour': range(8, 21)})
all_facilities = df_filtered['FacilityEnglish'].unique()

full_df = pd.DataFrame()
for facility in all_facilities:
    facility_data = pd.merge(all_hours, df_filtered[df_filtered['FacilityEnglish'] == facility], on='Hour', how='left')
    facility_data['FacilityEnglish'] = facility
    # 對該設施的 StandbyTime 進行線性插值和前後填充
    facility_data['StandbyTime'] = facility_data['StandbyTime'].interpolate(method='linear').ffill().bfill()
    full_df = pd.concat([full_df, facility_data], ignore_index=True)

# 計算每個設施在 8 點到 20 點之間的平均等待時間
average_wait_times = full_df.groupby('FacilityEnglish')['StandbyTime'].mean().reset_index()

# 顯示結果
print(average_wait_times)

In [ ]:
import matplotlib.pyplot as plt

# 設置圖表大小
plt.figure(figsize=(24, 18))

# 為每個設施畫折線
for facility in full_df['FacilityEnglish'].unique():
    facility_data = full_df[full_df['FacilityEnglish'] == facility]
    plt.plot(facility_data['Hour'], facility_data['StandbyTime'], marker='o', label=facility)

# 添加圖例和標籤
plt.legend(title='Facility')
plt.xlabel('Hour')
plt.ylabel('Average Standby Time (minutes)')
plt.title('Average Standby Time by Hour')
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 設置圖表大小
plt.figure(figsize=(18, 12))

# 創建透視表（pivot table）使用full_df，而非average_wait_times
pivot_df = full_df.pivot_table(index='Hour', columns='FacilityEnglish', values='StandbyTime', aggfunc='mean')

# 使用 Seaborn 繪製熱圖
sns.heatmap(pivot_df, annot=True, fmt=".1f", cmap="YlGnBu")

# 添加標題和標籤
plt.title('Heatmap of Average Standby Time by Hour and Facility')
plt.xlabel('Facility')
plt.ylabel('Hour')
plt.show()


In [ ]:
df_filtered.describe()

In [ ]:
df_filtered.shape

In [ ]:
df_filtered.isnull().sum()

In [ ]:
df_filtered.info()